In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils

# import other libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
import pickle
from typing import *
import time
import math
import random
import wandb
wandb.login()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: mantra7. Use `wandb login --relogin` to force relogin


In [2]:
# define Lang
class Lang:
	def __init__(self, wordList):
		self.char2index = {'A': 0, 'Z': 1, '_': 2, '^':3}
		self.char2count = {}
		self.index2char = {0: 'A', 1: 'Z', 2: '_', 3: '^'}
		self.n_chars = 4

		for word in wordList:
			self.addWord(word)

	def addWord(self, word):
		for char in word:
			self.addChar(char)

	def addChar(self, char):
		if char not in self.char2index:
			self.char2index[char] = self.n_chars
			self.char2count[char] = 1
			self.index2char[self.n_chars] = char
			self.n_chars += 1
		else:
			self.char2count[char] += 1

	def encode(self, word):
		encoded = [0] * len(word)
		for i in range(len(word)):
			if word[i] in self.char2index:
				encoded[i] = self.char2index[word[i]]
			else:
				encoded[i] = self.char2index['^']
		return encoded
	
	def one_hot_encode(self, word):
		one_hot = torch.zeros(len(word), self.n_chars, device=device)
		for i in range(len(word)):
			if word[i] in self.char2index:
				one_hot[i][self.char2index[word[i]]] = 1
			else:
				one_hot[i][self.char2index['^']] = 1			
		return one_hot
		
	def decode(self, word):
		decoded = ''
		for i in range(len(word)):
			if word[i].argmax().item() in self.index2char:
				decoded += self.index2char[word[i].argmax().item()]
			else:
				decoded += '^'
		return decoded
	
	def decode_one_hot(self, word):
		decoded = ''
		for i in range(len(word)):
			if word[i].argmax().item() in self.index2char:
				decoded += self.index2char[word[i].argmax().item()]
			else:
				decoded += '^'
		return decoded

In [3]:
def tensorFromWord(lang : Lang, word : str):
    indexes = lang.encode(word)
    indexes.append(1)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)

def tensorsFromPair(pair, inp_lang : Lang, out_lang : Lang):
    input_tensor = tensorFromWord(inp_lang, pair[0])
    target_tensor = tensorFromWord(out_lang, pair[1])
    return (input_tensor.unsqueeze(1), target_tensor)

In [4]:
# # create dataset
# class AksharantarDataset(Dataset):
# 	def __init__(self, data, inp_lang, out_lang):
# 		self.data = data
# 		self.inp_lang = inp_lang
# 		self.out_lang = out_lang

# 	def __len__(self):
# 		return len(self.data)

# 	def __getitem__(self, idx):
# 		if torch.is_tensor(idx):
# 			idx = idx.tolist()

# 		inp_seq = self.inp_lang.one_hot_encode(self.data['input_seq'][idx]).unsqueeze(1)
# 		out_seq = self.out_lang.one_hot_encode(self.data['target_seq'][idx]).unsqueeze(1)

# 		sample = {'input_seq': inp_seq, 'target_seq': out_seq}
# 		return sample

In [5]:
def DataLoader(lang : str):
	train_data = pd.read_csv(f'aksharantar_sampled/{lang}/{lang}_train.csv')
	test_data = pd.read_csv(f'aksharantar_sampled/{lang}/{lang}_test.csv')
	valid_data = pd.read_csv(f'aksharantar_sampled/{lang}/{lang}_valid.csv')
	
	train_data.columns = ['input_seq', 'target_seq']
	test_data.columns = ['input_seq', 'target_seq']
	valid_data.columns = ['input_seq', 'target_seq']

	return train_data, test_data, valid_data

def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

In [6]:
def get_cell(str):
	if str == 'lstm':
		return nn.LSTM
	elif str == 'gru':
		return nn.GRU
	elif str == 'rnn':
		return nn.RNN
	else:
		raise ValueError('Invalid cell type')

In [7]:
class EncoderRNN(nn.Module):
	def __init__(self, input_size, embed_size, hidden_size, n_layers=1, type='gru', dropout=0.2):
		super(EncoderRNN, self).__init__()
		self.hidden_size = hidden_size
		self.n_layers = n_layers
		self.type_t = type

		self.embedding = nn.Embedding(input_size, embed_size)
		self.cell = get_cell(type)(embed_size, hidden_size, n_layers, dropout=dropout)

	def forward(self, input, hidden):
		embedded = self.embedding(input)
		output = embedded
		output, hidden = self.cell(output, hidden)
		return output, hidden

	def initHidden(self):
		if self.type_t == 'lstm':
			return (torch.zeros(self.n_layers, 1, self.hidden_size, device=device), torch.zeros(self.n_layers, 1, self.hidden_size, device=device))
		return torch.zeros(self.n_layers, 1, self.hidden_size, device=device)
	
class DecoderRNN(nn.Module):
	def __init__(self, hidden_size, output_size, n_layers=1, type='gru', dropout=0.2):
		super(DecoderRNN, self).__init__()
		self.hidden_size = hidden_size
		self.n_layers = n_layers
		self.type_t = type

		self.embedding = nn.Embedding(output_size, hidden_size)
		self.cell = get_cell(type)(hidden_size, hidden_size, n_layers, dropout=dropout)
		self.out = nn.Linear(hidden_size, output_size)
		self.softmax = nn.LogSoftmax(dim=1)

	def forward(self, input, hidden):
		output = self.embedding(input).view(1, 1, -1)
		output = F.relu(output)
		if self.type_t == 'lstm':
			output, (hidden, cell) = self.cell(output, (hidden, cell))
		else:
			output, hidden = self.cell(output, hidden)
		output = self.softmax(self.out(output[0]))
		return output, hidden

	def initHidden(self):
		if self.type_t == 'lstm':
			return (torch.zeros(self.n_layers, 1, self.hidden_size, device=device), torch.zeros(self.n_layers, 1, self.hidden_size, device=device))
		return torch.zeros(self.n_layers, 1, self.hidden_size, device=device)


In [8]:
class Seq2Seq(nn.Module):
	def __init__(self, input_size, hidden_size, embed_size, output_size, n_layers=1, type='gru', dropout=0.2):
		super(Seq2Seq, self).__init__()
		self.input_size = input_size
		self.hidden_size = hidden_size
		self.output_size = output_size
		self.n_layers = n_layers

		self.encoder = EncoderRNN(input_size, embed_size, hidden_size, n_layers, type, dropout).to(device)
		self.decoder = DecoderRNN(hidden_size, output_size, n_layers, type, dropout).to(device)

	def forward(self, input_tensor, target_tensor, max_length=50):
		encoder_hidden = self.encoder.initHidden()

		input_length = input_tensor.size(0)
		target_length = target_tensor.size(0)

		encoder_outputs = torch.zeros(max_length, self.encoder.hidden_size, device=device)

		for ei in range(input_length):
			encoder_output, encoder_hidden = self.encoder(
				input_tensor[ei], encoder_hidden)
			encoder_outputs[ei] = encoder_output[0, 0]

		decoder_input = torch.tensor([[0]], device=device)  # SOS

		decoder_hidden = encoder_hidden
		use_teacher_forcing = True if random.random() < 0.5 else False

		decoder_outputs = []
		if use_teacher_forcing:
			# Teacher forcing: Feed the target as the next input
			for di in range(target_length):
				decoder_output, decoder_hidden = self.decoder(
					decoder_input, decoder_hidden)
				decoder_outputs.append(decoder_output)
				decoder_input = target_tensor[di]  # Teacher forcing
		else:
			# Without teacher forcing: use its own predictions as the next input
			for di in range(target_length):
				decoder_output, decoder_hidden = self.decoder(
					decoder_input, decoder_hidden)
				decoder_outputs.append(decoder_output)
				topv, topi = decoder_output.topk(1)
				decoder_input = topi.squeeze().detach()  # detach from history as input

				if decoder_input.item() == 1:
					break

		return decoder_outputs

	def predict(self, input_tensor, max_length = 50):
		encoder_hidden = self.encoder.initHidden()

		input_length = input_tensor.size(0)

		encoder_outputs = torch.zeros(max_length, self.encoder.hidden_size, device=device)

		for ei in range(input_length):
			encoder_output, encoder_hidden = self.encoder(
				input_tensor[ei], encoder_hidden)
			encoder_outputs[ei] = encoder_output[0, 0]

		decoder_input = torch.tensor([[0]], device=device)  # SOS

		decoder_outputs = []

		decoder_hidden = encoder_hidden
		for di in range(max_length):
			decoder_output, decoder_hidden = self.decoder(
				decoder_input, decoder_hidden)
			topv, topi = decoder_output.data.topk(1)
			if topi.item() == 1:
				break
			decoder_outputs.append(decoder_output)

			decoder_input = topi.squeeze().detach()
		
		return decoder_outputs

In [9]:
class Translator:
	def __init__(self, lang, embed_size=10, hidden_size=10, n_layers=1, max_length=50, type='gru', dropout=0.2):
		self.train_data, self.test_data, self.valid_data = DataLoader(lang)

		self.inp_lang = Lang(self.train_data['input_seq'])
		self.out_lang = Lang(self.train_data['target_seq'])

		self.model = Seq2Seq(self.inp_lang.n_chars, hidden_size, embed_size, self.out_lang.n_chars, n_layers, type, dropout)
		self.criterion = nn.NLLLoss()
		self.max_length = max_length

		self.pairs = [tensorsFromPair((self.train_data['input_seq'][i], self.train_data['target_seq'][i]), self.inp_lang, self.out_lang)
								for i in range(len(self.train_data))]

	def trainOne(self, input_tensor, target_tensor):
		self.encoder_optim.zero_grad()
		self.decoder_optim.zero_grad()

		decoder_outputs = self.model.forward(input_tensor, target_tensor, self.max_length)

		loss = 0
		for di in range(len(decoder_outputs)):
			loss += self.criterion(decoder_outputs[di], target_tensor[di])
		loss.backward()

		self.encoder_optim.step()
		self.decoder_optim.step()

		return loss.item() / target_tensor.size(0)

	def train(self,epoch=1, n_iters=10000, print_every=1000, plot_every=100, learning_rate=0.01, rand=False, dumpName='model', log=False, wandb = None):
		self.encoder_optim = optim.Adam(self.model.encoder.parameters(), lr=learning_rate)
		self.decoder_optim = optim.Adam(self.model.decoder.parameters(), lr=learning_rate)

		start = time.time()
		train_loss = []
		train_acc = []
		valid_loss = []
		valid_acc = []

		for i in range(epoch):
			print_loss_total = 0
			tot_loss = 0
			print("Epoch: ", i)
			if rand:
				training_pairs = [random.choice(self.pairs) for i in range(n_iters)]
			else:
				training_pairs = self.pairs

			for iter in tqdm(range(1, len(training_pairs) + 1)):
				training_pair = training_pairs[iter - 1]
				input_tensor = training_pair[0]
				target_tensor = training_pair[1]

				loss = self.trainOne(input_tensor, target_tensor)
				print_loss_total += loss 
				tot_loss += loss

				if iter % print_every == 0:
					print_loss_avg = print_loss_total / print_every
					print_loss_total = 0
					print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
												iter, iter / n_iters * 100, print_loss_avg))
			train_loss.append(tot_loss / len(training_pairs))
			train_acc.append(self.accuracy(self.train_data))
			valid_stats = self.calculate_stats(self.valid_data)
			valid_loss.append(valid_stats[0])
			valid_acc.append(valid_stats[1])
			print("Train Loss: ", train_loss[-1], "Valid Loss: ", valid_loss[-1], "Train Acc: ", train_acc[-1], "Valid Acc: ", valid_acc[-1])
			pickle.dump(self, open(dumpName + '_'+str(i) + '.pkl', 'wb'))
			if log:
				wandb.log({'train_loss': train_loss[i], 'train_accuracy': train_acc[i], 'val_loss': valid_loss[i], 'val_accuracy': valid_acc[i]})
		return train_loss, train_acc, valid_loss, valid_acc
					
	def accuracy(self, data):
		with torch.no_grad():
			return np.sum([(self.translate(data['input_seq'][i]) == data['target_seq'][i]) for i in range(len(data))]) / len(data)
				
	def translate(self, word):
		with torch.no_grad():
			tensor = tensorFromWord(self.inp_lang, word).unsqueeze(1)
			outs = self.model.predict(tensor, self.max_length)
			return self.out_lang.decode(outs)
	
	def calculate_stats(self, data):
		with torch.no_grad():
			loss = 0
			acc = 0
			for i in range(len(data)):
				tensor = tensorFromWord(self.inp_lang, data['input_seq'][i]).unsqueeze(1)
				output = self.model.predict(tensor, self.max_length)
				word = self.out_lang.decode(output)
				target = data['target_seq'][i]
				acc += (word == target)
				target = tensorFromWord(self.out_lang, data['target_seq'][i])

				mx_len = min(len(output), len(target))

				while(len(output) < mx_len):
					output = torch.cat((output, self.out_lang.encode('_')), 0, device=device)

				while(len(target) < mx_len):
					target = torch.cat((target, self.out_lang.encode('_')), 0, device=device)

				for di in range(mx_len):
					loss += self.criterion(output[di], target[di]) / mx_len
			return loss.item() / len(data), acc / len(data)

In [10]:
sweep_configuration_new = {
	'method': 'bayes',
	'metric': {'goal': 'maximize', 'name': 'val_accuracy'},
	'parameters': 
	{
		'epochs' : {'values': [10]},
		'lr' : {'values': [0.001, 0.005]},
		'hidden_size' : {'values': [512]},
        'embed_size' : {'values': [16, 32]},
        'n_layers' : {'values': [1]},
        'type' : {'values': ['lstm', 'gru']},
        'dropout' : {'values': [0, 0.2, 0.3]}
	}
}

In [11]:
f = open('configs', 'w')
count = 0

def get_name(config):
	global count
	count += 1
	f.write(str(count) + '_' + str(config) + '\n')
	f.flush()
	return 'lr'+str(config['lr'])+'_hidden_size'+str(config['hidden_size'])+'_embed_size'+str(config['embed_size'])+'_n_layers'+str(config['n_layers'])+'_type'+str(config['type'])+'_dropout'+str(config['dropout'])

# 1: Define objective/training function
def objective(config):
	translator = Translator('guj', hidden_size=config.hidden_size, embed_size=config.embed_size, n_layers=config.n_layers, type=config.type, dropout=config.dropout)
	dump = 'trans'+'_'+str(config.hidden_size)+'_'+str(config.embed_size)+'_'+str(config.n_layers)+'_'+str(config.type)+'_'+str(config.dropout)
	train_loss, train_acc, valid_loss, valid_acc = translator.train(epoch=config.epochs, print_every=10000, learning_rate=config.lr, dumpName=dump, log=True, wandb=wandb)

	# for i in range(config.epochs):
	# 	wandb.log({'train_loss': train_loss[i], 'train_accuracy': train_acc[i], 'val_loss': valid_loss[i], 'val_accuracy': valid_acc[i]})

def main():
	run = wandb.init(project='rnn-first-proj')
	objective(wandb.config)
	run.name = get_name(wandb.config)
	run.finish()

# 2: Define the search space
sweep_configuration = sweep_configuration_new

# 3: Start the sweep
sweep_id = wandb.sweep(sweep=sweep_configuration, project='rnn-first-proj')
wandb.agent(sweep_id, function=main, count=10)

f.close()

Create sweep with ID: ohi6vwf6
Sweep URL: https://wandb.ai/mantra7/rnn-first-proj/sweeps/ohi6vwf6


wandb: Agent Starting Run: 8ggf93de with config:
wandb: 	dropout: 0
wandb: 	embed_size: 32
wandb: 	epochs: 10
wandb: 	hidden_size: 512
wandb: 	lr: 0.001
wandb: 	n_layers: 1
wandb: 	type: gru
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
C:\Users\HP\AppData\Roaming\Python\Python39\site-packages\IPython\html.py:12: ShimWarning: The `IPython.html` package has been deprecated since IPython 4.0. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  warn("The `IPython.html` package has been deprecated since IPython 4.0. "
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


Epoch:  0


 20%|█▉        | 10001/51199 [15:15<57:26, 11.95it/s] 

15m 15s (- 0m 0s) (10000 100%) 1.8664


...

 59%|█████▊    | 30001/51199 [45:05<31:20, 11.27it/s]

429m 30s (- -287m 39s) (30000 300%) 0.8814


 78%|███████▊  | 40001/51199 [1:00:24<35:29,  5.26it/s]

444m 49s (- -334m 23s) (40000 400%) 0.8709


 97%|█████████▋| 49566/51199 [1:18:16<02:34, 10.55it/s]

In [ ]:
# trans = Translator('guj', hidden_size=128, embed_size=32, n_layers=2, type='gru', dropout=0.2)
# trans.train(n_iters=1, print_every=1000, learning_rate=0.005, dumpName='lol', rand=True)

Epoch:  0


100%|██████████| 1/1 [00:00<00:00, 22.38it/s]


([4.174344635009765], [0.0], [4.133323794261294], [0.0])

In [ ]:
trans.translate('a')

'સરાાાાા'

In [ ]:
# # create a seq2seq model using 2 RNNs
# class Seq2Seq(nn.Module):
# 	def __init__(self, input_size, hidden_size, output_size, n_layers=1):
# 		super(Seq2Seq, self).__init__()
# 		self.input_size = input_size
# 		self.hidden_size = hidden_size
# 		self.output_size = output_size
# 		self.n_layers = n_layers

# 		# encoder and decoder
# 		self.encoder = nn.RNN(input_size, hidden_size, n_layers)
# 		self.decoder = nn.RNN(hidden_size, hidden_size, n_layers)

# 		# linear layer to get output
# 		self.linear = nn.Linear(hidden_size, output_size)

# 	def forward(self, input, hidden):
# 		# encoder
# 		output, hidden = self.encoder(input, hidden)
		
# 		# decoder
# 		output, hidden = self.decoder(output, hidden)
		
# 		# get output
# 		output = self.linear(output)
# 		return output, hidden
	
# 	def predict(self, input, inp_lang, out_lang):
# 		out, hidden = self.forward(inp_lang.one_hot_encode(input).unsqueeze(1), self.init_hidden(1))
# 		return out_lang.decode_one_hot(out)
	
# 	def init_hidden(self, batch_size):
# 		return Variable(torch.zeros(self.n_layers, batch_size, self.hidden_size))

In [ ]:
# class Translator:
# 	def __init__(self, lang):
# 		train_data = pd.read_csv(f'aksharantar_sampled/{lang}/{lang}_train.csv')
# 		test_data = pd.read_csv(f'aksharantar_sampled/{lang}/{lang}_test.csv')
# 		valid_data = pd.read_csv(f'aksharantar_sampled/{lang}/{lang}_valid.csv')

# 		train_data.columns = ['input_seq', 'target_seq']
# 		test_data.columns = ['input_seq', 'target_seq']
# 		valid_data.columns = ['input_seq', 'target_seq']

# 		self.inp_lang = Lang(train_data['input_seq'])
# 		self.out_lang = Lang(train_data['target_seq'])

# 		self.model = Seq2Seq(self.inp_lang.n_chars, 10, self.out_lang.n_chars, 1)
# 		self.criterion = nn.CrossEntropyLoss()
# 		self.optimizer = optim.Adam(self.model.parameters(), lr=0.001)

# 		train_dataset = AksharantarDataset(train_data, self.inp_lang, self.out_lang)
# 		test_dataset = AksharantarDataset(test_data, self.inp_lang, self.out_lang)
# 		valid_dataset = AksharantarDataset(valid_data, self.inp_lang, self.out_lang)

# 		self.train_dataloader = DataLoader(train_dataset, batch_size=1, shuffle=True)
# 		self.test_dataloader = DataLoader(test_dataset, batch_size=1, shuffle=True)
# 		self.valid_dataloader = DataLoader(valid_dataset, batch_size=1, shuffle=True)

# 	def translate(self, word):
# 		return self.model.predict(word, self.inp_lang, self.out_lang)
	
# 	def train_one(self, inp, target):
# 		# zero gradients
# 		self.optimizer.zero_grad()
		
# 		# initialize hidden layer
# 		hidden = self.model.init_hidden(1)
		
# 		# get output
# 		output, hidden = self.model.forward(inp, hidden)
		
# 		mx_len = min(len(output), len(target))

# 		if(len(output) != len(inp)):
# 			print('lol')

# 		# append output and target with 'Z' to make them of mx_len
# 		while(len(output) < mx_len):
# 			output = torch.cat((output, self.out_lang.one_hot_encode_char('Z')), 0)

# 		while(len(target) < mx_len):
# 			target = torch.cat((target, self.out_lang.one_hot_encode_char('Z')), 0)
		
# 		# calculate loss 
# 		loss = self.criterion(torch.flatten(output[:mx_len], 0, 1), torch.flatten(target[:mx_len], 0, 1).max(1)[1])
			
# 		# backpropagate
# 		loss.backward()
		
# 		# update weights
# 		self.optimizer.step()

# 		return loss.data.item() / len(output)
	
# 	def train_epoch(self, data_loader, print_every=100):
# 		loss = 0
# 		for i_batch, sample_batched in tqdm(enumerate(data_loader)):
# 			loss += self.train_one(sample_batched['input_seq'][0], sample_batched['target_seq'][0])
# 			if i_batch % print_every == print_every-1:
# 				print(' Train Loss: ', loss / (i_batch+1))
# 				# print(' Valid Loss: ', self.calc_loss_full(self.valid_dataloader))
# 				# print(' Train Accuracy: ', self.calc_accuracy(self.train_dataloader))
# 				# print(' Valid Accuracy: ', self.calc_accuracy(self.valid_dataloader))
# 		return self.calc_loss_full(data_loader)
	
# 	def train(self, epochs, print_every=100):
# 		losses = []
# 		for epoch in range(epochs):
# 			print('Epoch ', epoch + 1)
# 			loss = self.train_epoch(self.train_dataloader, print_every)
# 			losses.append(loss)
# 		return losses
	
# 	def calc_loss_full(self, data_loader):
# 		loss = 0
# 		for i_batch, sample_batched in tqdm(enumerate(data_loader)):
# 			inp = sample_batched['input_seq'][0]
# 			target = sample_batched['target_seq'][0]
# 			hidden = self.model.init_hidden(1)
# 			output, hidden = self.model.forward(inp, hidden)
# 			mx_len = min(len(output), len(target))
# 			while(len(output) < mx_len):
# 				output = torch.cat((output, self.out_lang.one_hot_encode_char('Z')), 0)
# 			while(len(target) < mx_len):
# 				target = torch.cat((target, self.out_lang.one_hot_encode_char('Z')), 0)
# 			loss += self.criterion(torch.flatten(output[:mx_len], 0, 1), torch.flatten(target[:mx_len], 0, 1).max(1)[1])
# 		return loss / len(data_loader)
	
# 	def calc_accuracy(self, dataset):
# 		return np.sum([(self.translate(sample_batched['input_seq']) ==  sample_batched['target_seq']) for sample_batched in dataset]) / len(dataset)

In [ ]:
# guj_trans = Translator('guj')
# guj_trans.train(10, 1000)